# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [2]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, N):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.gru = torch.nn.GRU(
            input_size=N+1,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )
    self.fc = torch.nn.Linear(hidden_size, N+1)
    self.softmax = torch.nn.Softmax(dim=2)


  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    x, h = self.gru(x)
    x = self.fc(x)
    x = self.softmax(x)
    return x

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    x = idx_to_onehot(torch.tensor(list(ord(c)-96 for c in s)).view(1,-1))
    x = x.to(device)
    x = self.forward(x)
    x = x[0].max(1, keepdim=True)[1].view(-1)
    ss = ''
    for n in x:
      n = n.item()
      if n==0:
        ss += ' '
      else:
        ss += chr(n+96)
    return ss

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [3]:
def test_model(model, sequence_length=15, D=4):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    assert D > 0, 's[:-D] won\'t work for D=0'
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [4]:
BATCH_SIZE = 5
ds_load= torch.utils.data.DataLoader(ds, batch_size= BATCH_SIZE)

In [5]:
import time

start_time = time.time()

# TODO: initialize and train your model here.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50
model = GRUMemory(hidden_size, N).to(device)
        
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)


n_epochs = 5
epoch = 0
num_batch = 10000

while epoch < n_epochs:
    model.train()
    batch = 0
    total_loss = 0
    for seq, result in ds_load:
      data, target = idx_to_onehot(seq).to(device), idx_to_onehot(result).to(device)
      optimizer.zero_grad()
      logits = model(data)
      loss = criterion(logits, target)
      total_loss += loss.item()
      loss.backward()
      optimizer.step()
      batch += 1
      if batch >= num_batch:
        break
    # print(total_loss/num_batch/BATCH_SIZE)
    epoch += 1

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model,DELAY)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [6]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

MAX_DELAY = 8
SEQ_LENGTH = 20

ds_delay = VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size = DATASET_SIZE)

### Model

And the model.

In [7]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay,N):
    super().__init__()
    self.hidden_size = hidden_size
    self.max_delay = max_delay
    self.hidden_size = hidden_size
    self.gru = torch.nn.GRU(
            input_size=N+1+self.max_delay+1,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )
    self.fc = torch.nn.Linear(hidden_size, N+1)
    self.softmax = torch.nn.Softmax(dim=2)


  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    delays = self.idx_to_onehot(delays.view(-1,1),self.max_delay+1).to(device)
    delays = delays.repeat(1,x.size(1),1)
    x= torch.cat((x,delays), dim=2)
    x, h = self.gru(x)
    x = self.fc(x)
    x = self.softmax(x)
    return x

  def idx_to_onehot(self, x, k):
    ones = torch.sparse.torch.eye(k).to(device)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64)).to(device)
    return res.view(*shape, res.shape[-1]).to(device)


  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    x = idx_to_onehot(torch.tensor(list(ord(c)-96 for c in s)).view(1,-1))
    x = x.to(device)
    delay = torch.tensor([delay]).to(device)
    x = self.forward(x, delay)
    x = x[0].max(1, keepdim=True)[1].view(-1)
    ss = ''
    for n in x:
      n = n.item()
      if n==0:
        ss += ' '
      else:
        ss += chr(n+96)
    return ss


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [8]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [9]:
import time

BATCH_SIZE = 10
delay_load= torch.utils.data.DataLoader(ds_delay, batch_size= BATCH_SIZE)

start_time = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 100
model_delay = VariableDelayGRUMemory(hidden_size,MAX_DELAY, N).to(device)
        
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model_delay.parameters(), lr=0.001)


n_epochs = 5
epoch = 0
num_batch = 20000

while epoch < n_epochs:
    model_delay.train()
    batch = 0
    total_loss = 0
    for seq, delay, result in delay_load:
      data, target = idx_to_onehot(seq).to(device), idx_to_onehot(result).to(device)
      delay = delay.to(device)
      optimizer.zero_grad()
      logits = model_delay(data, delay)
      loss = criterion(logits, target)
      total_loss += loss.item()
      loss.backward()
      optimizer.step()
      batch += 1
      if batch >= num_batch:
        break
    # print(total_loss/num_batch/BATCH_SIZE)
    epoch += 1


end_time = time.time()
accuracy = test_variable_delay_model(model_delay)
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert accuracy > 0.99, 'accuracy is too low'
print('tests passed')

tests passed
